In [1]:

!pip install "numpy>=1.12.1"
!pip install "scipy>=1.0.1"
!pip install "torch>=2.0.0"
!pip install accelerate==0.23.0
!pip install bitsandbytes
!pip install peft==0.5.0
!pip install transformers==4.34.0
!pip install datasets


  Obtaining dependency information for accelerate==0.23.0 from https://files.pythonhosted.org/packages/d9/92/2d3aecf9f4a192968035880be3e2fc8b48d541c7128f7c936f430d6f96da/accelerate-0.23.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 1.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.24.1
    Uninstalling accelerate-0.24.1:
      Successfully uninstalled accelerate-0.24.1
  Obtaining dependency information for bitsandbytes from https://files.pythonhosted.org/packages/e4/e6/ccb84da7ffaf208a71c2c3c8e1120b34759df640db959660be9a98505eb4/bitsandbytes-0.44.1-py3-none-manylinux_2_24_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 9.7 MB/s eta 0:00:00:00:0100:01
  Obtaining dependency information for peft==0.5.0 from https://files.pythonhosted.org/packages/37/1a/8d20e8704da9fa070eb909265584b960da57be1d833d550c59f50906dc5c/peft-0.5.0-py3-none-any.whl.metadata
   ━━━━

'\n!pip install zstandard\n!pip install jsonlines\n!pip install sentencepiece\n!pip install fire\n!pip install datasketch==1.5.9\n!pip install nltk==3.8.1\n!pip install scikit-learn==1.3.0\n!pip install pytest'

In [2]:
import torch
import pandas as pd
from peft import PeftModel, PeftConfig
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForTokenClassification, AutoConfig, GenerationConfig
from transformers import Trainer, TrainingArguments, logging, TrainerCallback, TrainerState, TrainerControl, BitsAndBytesConfig
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training
import torch.nn.functional as F
from datasets import load_dataset
import time
from typing import Any, List, Mapping, Optional
import transformers
import os
from pathlib import Path

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### Загружаем датасет

In [4]:
from datasets import load_dataset

In [5]:
data = load_dataset(
    "json", 
    data_files={'train' : '/kaggle/input/fine-tunning-llama/train.json' , 'validation' : '/kaggle/input/fine-tunning-llama/val.json'}
)

data

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e727458344e87341/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 22
    })
    validation: Dataset({
        features: ['system', 'user', 'bot'],
        num_rows: 6
    })
})

## Предобработка датасета

In [ ]:
print(generate_and_tokenize_prompt(data["train"][0]))

In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel, PeftConfig

MODEL_NAME = "IlyaGusev/saiga2_7b_lora"

# Определяем устройство
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Загружаем конфигурацию адаптеров
config = PeftConfig.from_pretrained(MODEL_NAME)

# Загружаем базовую модель с квантованием и указываем точное устройство
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,  # Загружаем в 8-битной точности
    torch_dtype=torch.float16,  # Используем float16 для экономии памяти
    device_map={'': torch.cuda.current_device()}  # Явно указываем устройство
)

# Подключаем адаптеры для тренировки
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16,
    is_trainable=True  # Делаем модель обучаемой
)

# Переводим модель в режим тренировки
model.train()

# Загружаем токенизатор и конфигурацию генерации
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)

# Печатаем конфигурацию генерации
print(generation_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 3584,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.2,
  "temperature": 0.5,
  "top_k": 30,
  "top_p": 0.9
}



In [12]:
train_data = (
    data["train"].map(generate_and_tokenize_prompt)
)

val_data = (
    data["validation"].map(generate_and_tokenize_prompt)
)

  0%|          | 0/22 [00:00<?, ?ex/s]

  0%|          | 0/6 [00:00<?, ?ex/s]

## Обучаем модель

In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [14]:
BATCH_SIZE = 4
MICRO_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 100
OUTPUT_DIR = "/kaggle/working/tmp"

training_arguments = transformers.TrainingArguments(
            per_device_train_batch_size=MICRO_BATCH_SIZE,
            gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
#             warmup_steps=200,
            max_steps=TRAIN_STEPS,
            learning_rate=LEARNING_RATE,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="steps",
            save_strategy="steps",
            eval_steps=10,
            save_steps=10,
            output_dir=OUTPUT_DIR,
            save_total_limit=10,
            load_best_model_at_end=True,
            report_to=None,
            overwrite_output_dir=True, # Overwrite the content of the output dir
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
)

In [21]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

In [22]:
torch.cuda.empty_cache()


In [16]:
# Настройка тренера и передача модели в него
trainer = transformers.Trainer(
    model=model,  # Модель для тренировки
    train_dataset=train_data,  # Тренировочный датасет
    eval_dataset=val_data,  # Валидационный датасет
    args=training_arguments,  # Аргументы для тренировки
    data_collator=data_collator  # Коллатор данных
)

# Запуск тренировки
trainer.train()

# Сохранение обученной модели
model.save_pretrained(OUTPUT_DIR)


Step,Training Loss,Validation Loss
10,11.979400,nan
20,8.297400,nan
30,5.052200,nan
40,18.143000,nan
50,18.915000,nan
60,15.772500,nan
70,9.750200,nan
80,15.809000,nan
90,12.240600,nan
100,16.319600,nan


In [21]:
SYSTEM_PROMPT = "Ты извлекаешь термины и определения из текста"
SYSTEM_TOKEN = 1788
USER_TOKEN = 1404
BOT_TOKEN = 9225
LINEBREAK_TOKEN = 13

top_k=40
top_p=0.5
temperature=0.01
repeat_penalty=1.1


ROLE_TOKENS = {
    "user": USER_TOKEN,
    "bot": BOT_TOKEN,
    "system": SYSTEM_TOKEN
}


def get_message_tokens(model, role, content):
    message_tokens = model.tokenize(content.encode("utf-8"))
    message_tokens.insert(1, ROLE_TOKENS[role])
    message_tokens.insert(2, LINEBREAK_TOKEN)
    message_tokens.append(model.token_eos())
    return message_tokens


def get_system_tokens(model):
    system_message = {
        "role": "system",
        "content": SYSTEM_PROMPT
    }
    return get_message_tokens(model, **system_message)

def chat_saiga(message, model):
    system_tokens = get_system_tokens(model)
    tokens = system_tokens
    # model.eval(tokens)
    
    message_tokens = get_message_tokens(model=model, role="user", content=message)
    role_tokens = [model.token_bos(), BOT_TOKEN, LINEBREAK_TOKEN]
    tokens += message_tokens + role_tokens
    # print(tokens)
    # detokenize = model.detokenize(tokens)
    # print(model.tokenize(full_prompt))
    generator = model.generate(
        tokens,
        top_k = top_k,
        top_p = top_p,
        temp = temperature,
        repeat_penalty = repeat_penalty,
        reset = True
    )
    # print(len([token for token in generator]))
    
    result_list = []
    
    for token in generator:
        token_str = model.detokenize([token]).decode("utf-8", errors="ignore")
        tokens.append(token)
        if token == model.token_eos():
            break
        print(token_str, end="", flush=True)
        result_list.append(token_str)
    return ''.join(result_list)

## Проверяем модель на адекватность

In [8]:
CUTOFF_LEN = 3584

def generate_prompt(data_point):
    promt = f"""<s>system
{data_point['system']}</s><s>user
{data_point['user']}</s><s>bot
{data_point['bot']}</s>"""
    return promt
 
    
def tokenize (prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)
        
        
    
    result["labels"] = result["input_ids"].copy()

    return result
 
def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [22]:
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты отвечаешь на вопрос пользователя"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
class Conversation:
    def __init__(
        self,
        message_template = DEFAULT_MESSAGE_TEMPLATE,
        system_prompt = DEFAULT_SYSTEM_PROMPT,
        response_template = DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()



def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt,
                     return_tensors="pt",
                     add_special_tokens=False,
#                      padding=True,
#                     truncation=True
                    )
    data = {k: v.to(device) for k, v in data.items()}
    
    output_ids = model.generate(
        **data,
        generation_config = generation_config,
#         remove_invalid_values = True
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [23]:
%%time
def get_message(inputs):
    result = []
    for inp in inputs:
        conversation = Conversation()
        conversation.add_user_message(inp)
        prompt = conversation.get_prompt(tokenizer)
        print(prompt)
        output = generate(model, tokenizer, prompt, generation_config)
        result.append(output)
    return result

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 10.7 µs


In [21]:
p = """
Извлеки из данного текста все определения и термины.
Вывод должен быть в виде списка словарей с ключами term и definition

Линейная модель — модель, отображающая состояние или функционирование системы таким образом, что все взаимозависимости в ней принимаются линейными (см. Линейная зависимость, Линейность в экономике). Соответственно, она может формулироваться в виде одного линейного уравнения или системы линейных уравнений.
Линейное пространство это математическая структура, представляющая собой набор элементов, называемых векторами, а яблоко это фрукт
"""
get_message([p])

<s>system
Ты извлекаешь термины и определения из текста</s><s>user

Извлеки из данного текста все определения и термины.
Вывод должен быть в виде списка словарей с ключами term и definition

Линейная модель — модель, отображающая состояние или функционирование системы таким образом, что все взаимозависимости в ней принимаются линейными (см. Линейная зависимость, Линейность в экономике). Соответственно, она может формулироваться в виде одного линейного уравнения или системы линейных уравнений.
Линейное пространство это математическая структура, представляющая собой набор элементов, называемых векторами, а яблоко это фрукт
</s><s>bot


['term: Линейная модель\ndefinition: Модель, отображающая состояние или функционировку системы таким образом, что все взаимодействия между ее компонентами принимают линейные характер (см. Линейный анализ, Линейно-преобразованное пространство), соответственно можно выразить её одним линейным уравнением или системой линейных уравнений. \n\nterm: Линейное пространство\ndefinition: Математическое пространство, которое является подпространством стандартного Euclidean space R^n для некоего n >= 1 и обладает свойствами линейности относительно операции сложения векторов над этим пространством. Этот тип пространств широко используется в различных дисциплинах, включая алгебру, теорию чисел, геометрию, топологию и другие. Они также играют важную роль в теории матриц и других объектах, связанных с линейной алгебре.\n\nterm: Яблоко\ndefinition: Фрукты семейства Rosaceae, которые имеют мягкий плод без косточек, который обычно съедобен. Включает более чем тысячи видов, распространенных по всей Северн